# 日本語とコードをマージ

In [1]:
import math

## 各言語の語彙を読み込む

In [20]:
with open('./japanese.vocab') as f:
    japanese_vocab = list(filter(lambda x:len(x)==2, map(lambda x:x.split('\t'), f.read().strip().split('\n'))))
with open('./code.vocab') as f:
    code_vocab = list(filter(lambda x:len(x)==2, map(lambda x:x.split('\t'), f.read().strip().split('\n'))))

In [21]:
for v in japanese_vocab:
    v[1] = float(v[1])
for v in code_vocab:
    v[1] = float(v[1])

In [22]:
special_vocab = dict(filter(lambda x: x[1] == 0, sorted(japanese_vocab, key=lambda x:x[0])))
japanese_vocab = dict(filter(lambda x: x[1] != 0, sorted(japanese_vocab, key=lambda x:x[0])))
code_vocab = dict(filter(lambda x: x[1] != 0, sorted(code_vocab, key=lambda x:x[0])))

In [23]:
special_vocab['\n'] : 0.0

In [24]:
# 日本語とコードの比率は 7:1 程度．

## 確率を調整して語彙をマージ

In [25]:
all_vocab = {}

# for t, p in special_vocab.items():
#     all_vocab[t] = p

for t, p in japanese_vocab.items():
    all_vocab[t] = p + math.log(7/8)

for t, p in code_vocab.items():
    if t not in all_vocab:
        all_vocab[t] = p + math.log(1/8)
    else:
        all_vocab[t] = math.log(math.exp(p + math.log(1/8)) + math.exp(all_vocab[t]))

In [26]:
P = 0
for t, p in all_vocab.items():
    P += math.exp(p)
P

0.9941953655852239

In [27]:
all_vocab = sorted(list(all_vocab.items()), key=lambda x:x[0])

In [28]:
all_vocab[:200]

[('!', -6.514357077257914),
 ('!!', -8.552822579732048),
 ('!!!', -10.226643593722958),
 ('!!!!', -11.899231392624523),
 ('!!!!!', -12.290731392624522),
 ('!!」', -11.393431392624523),
 ('!!︎', -12.863131392624522),
 ('!!️', -12.394931392624523),
 ('!?', -9.912491392624522),
 ('!?」', -12.332231392624523),
 ('!」', -9.022391392624522),
 ('!』', -10.575831392624522),
 ('!【', -11.816131392624523),
 ('!】', -11.716731392624522),
 ('"', -6.3098584126173485),
 ('""', -12.284431392624523),
 ('"""', -9.344411541679836),
 ('"#', -11.325791541679836),
 ('"#!"', -14.705341541679836),
 ('"#"', -13.622541541679837),
 ('"#">', -13.286441541679837),
 ('"$', -13.491341541679835),
 ('"%', -11.815461541679836),
 ('",', -9.068037360427505),
 ('","', -9.977230848998115),
 ('","\\', -15.096241541679836),
 ('":', -10.387931392624523),
 ('":"', -10.029711168949317),
 ('":"","', -11.976431392624523),
 ('";', -10.503231541679837),
 ('">', -8.13570195470334),
 ('">#', -12.278360280747242),
 ('">$', -14.893941541679

In [29]:
with open('merged.vocab', 'w') as f:
    for t in sorted(special_vocab):
        f.write(f'{t}\t{0}\n')
    for t, p in all_vocab:
        f.write(f'{t}\t{p}\n')

In [30]:
len(japanese_vocab)

39600

In [31]:
len(code_vocab)

18600

In [32]:
len(all_vocab)

54992

In [33]:
count_code = 0
for t in code_vocab:
    if t not in japanese_vocab:
        count_code += 1
count_code

15392

In [34]:
15392 + 39600 + 400

55392

## vocab -> model
- 適当なベースモデルを作り，それをベースに作った語彙を突っ込む
- もしかするとここで `<pad>` を指定しておくべきだったか．

In [56]:
! python3 ./llm-jp-tokenizer/scripts/createModel.py \
    --data ./merged.vocab \
    --vocabSize 4835 \
    --maxLength 1 \
    --prefix dummy

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./merged.vocab
  input_format: 
  model_prefix: dummy
  model_type: UNIGRAM
  vocab_size: 4835
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 72
  num_sub_iterations: 2
  max_sentencepiece_length: 1
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 0
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  user_defined_symbols: 

  user_defined_symbols: <pad>
  user_defined_symbols: <CLS>
  user_defined_symbols: <SEP>
  user_defined_symbols: <EOD>
  user_defined_symbols: <MASK>
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 1
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_

LLM-jp tokenizer のコードを使って語彙からモデルを作る．

In [64]:
! python3 ./llm-jp-tokenizer/scripts/vocab2model.py \
    --vocab merged.vocab \
    --output merged.model \
    --baseModel dummy.model

## 使ってみる

In [1]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.Load("merged.model")

print(sp.EncodeAsPieces('こんにちは世界!'))
print(sp.EncodeAsIds('こんにちは世界!'))
print(sp.DecodeIds(sp.EncodeAsIds('こんにちは世界!')))

['▁こんにちは', '世界', '!']
[17940, 33450, 400]
こんにちは世界!


In [2]:
import termcolor
from termcolor import colored
print(colored('hello', 'red'), colored('world', 'green'))
print(colored("hello red world", 'red'))

hello world
hello red world


In [3]:
import random

In [4]:
import unicodedata

text = '''
<tanka>
    <input>マーチャーシュは税の基盤を領地から家庭に変え、時には戦時中での2年に1回に王税を集めることで農民の税を次第に増やしていった。</input>
    <bunsetsu>マーチャーシュは/税の/基盤を/領地から/家庭に/変え/時には/戦時中での/2年に/1回に/王/税を/集める/ことで/農民の/税を/次第に/増やして/いった</bunsetsu>
    <yomi>マ/ー/チャ/ー/シュ/ワ/ゼ/イ/ノ/キ/バ/ン/ヲ/リョ/ー/チ/カ/ラ/カ/テ/イ/ニ/カ/エ/ト/キ/ニ/ワ/セ/ン/ジ/チュ/ー/デ/ノ/ニ/ネ/ン/ニ/イ/ッ/カ/イ/ニ/オ/ー/ゼ/イ/ヲ/ア/ツ/メ/ル/コ/ト/デ/ノ/ー/ミ/ン/ノ/ゼ/イ/ヲ/シ/ダ/イ/ニ/フ/ヤ/シ/テ/イ/ッ/タ</yomi>
    <count>マ[1]ー[2]チャ[3]ー[4]シュ[5]ワ[6]ゼ[7]イ[8]ノ[9]キ[10]バ[11]ン[12]ヲ[13]リョ[14]ー[15]チ[16]カ[17]ラ[18]カ[19]テ[20]イ[21]ニ[22]カ[23]エ[24]ト[25]キ[26]ニ[27]ワ[28]セ[29]ン[30]ジ[31]チュ[32]ー[33]デ[34]ノ[35]ニ[36]ネ[37]ン[38]ニ[39]イ[40]ッ[41]カ[42]イ[43]ニ[44]オ[45]ー[46]ゼ[47]イ[48]ヲ[49]ア[50]ツ[51]メ[52]ル[53]コ[54]ト[55]デ[56]ノ[57]ー[58]ミ[59]ン[60]ノ[61]ゼ[62]イ[63]ヲ[64]シ[65]ダ[66]イ[67]ニ[68]フ[69]ヤ[70]シ[71]テ[72]イ[73]ッ[74]タ[75]</count>
    <output>オーゼイヲ/アツメルコトデ/ノーミンノ/ゼイヲシダイニ/フヤシテイッタ 王税を/集めることで/農民の/税を次第に/増やしていった</output>
</tanka>

'''

for s in sp.EncodeAsPieces(unicodedata.normalize('NFKC', text)):
    print(colored(s, random.choice(list(termcolor.COLORS.keys()))), end='')

▁
<tanka>
▁▁▁▁<input>マーチャーシュは税の基盤を領地から家庭に変え、時には戦時中での2年に1回に王税を集めることで農民の税を次第に増やしていった。</input>
▁▁▁▁<bunsetsu>マーチャーシュは/税の/基盤を/領地から/家庭に/変え/時には/戦時中での/2年に/1回に/王/税を/集める/ことで/農民の/税を/次第に/増やして/いった</bunsetsu>
▁▁▁▁<yomi>マ/ー/チャ/ー/シュ/ワ/ゼ/イ/ノ/キ/バ/ン/ヲ/リョ/ー/チ/カ/ラ/カ/テ/イ/ニ/カ/エ/ト/キ/ニ/ワ/セ/ン/ジ/チュ/ー/デ/ノ/ニ/ネ/ン/ニ/イ/ッ/カ/イ/ニ/オ/ー/ゼ/イ/ヲ/ア/ツ/メ/ル/コ/ト/デ/ノ/ー/ミ/ン/ノ/ゼ/イ/ヲ/シ/ダ/イ/ニ/フ/ヤ/シ/テ/イ/ッ/タ</yomi>
▁▁▁▁<count>マ[1]ー[2]チャ[3]ー[4]シュ[5]ワ[6]ゼ[7]イ[8]ノ[9]キ[10]バ[11]ン[12]ヲ[13]リョ[14]ー[15]チ[16]カ[17]ラ[18]カ[19]テ[20]イ[21]ニ[22]カ[23]エ[24]ト[25]キ[26]ニ[27]ワ[28]セ[29]ン[30]ジ[31]チュ[32]ー[33]デ[34]ノ[35]ニ[36]ネ[37]ン[38]ニ[39]イ[40]ッ[41]カ[42]イ[43]ニ[44]オ[45]ー[46]ゼ[47]イ[48]ヲ[49]ア[50]ツ[51]メ[52]ル[53]コ[54]ト[55]デ[56]ノ[57]ー[58]ミ[59]ン[60]ノ[61]ゼ[62]イ[63]ヲ[64]シ[65]ダ[66]イ[67]ニ[68]フ[69]ヤ[70]シ[71]テ[72]イ[73]ッ[74]タ[75]</count>
▁▁▁▁<output>オーゼイヲ/アツメルコトデ/ノーミンノ/ゼイヲシダイニ/フヤシテイッタ▁王税を/集めることで/農民の/税を次第に/増やしていった</output>
</tanka>



In [5]:
sp.Encode('<s>')

[16909, 267, 14080, 527]

In [6]:
sp.piece_to_id('<s>')

1

In [7]:
sp.EncodeAsPieces('</s>')

['▁', '<', '/', 's', '>']

In [13]:
import unicodedata

text = '''
<tanka>
'''.strip()

for s in sp.EncodeAsPieces(unicodedata.normalize('NFKC', text)):
    print(colored(s, random.choice(list(termcolor.COLORS.keys()))), end='')

▁<tanka>

In [5]:
import unicodedata

text = '''
[構文]
インデントが意味を持つ「オフサイドルール」が特徴的である。

以下に、階乗 (関数名: factorial)を題材にC言語と比較した例を示す。

Pythonのコード:

def factorial(x):
    if x == 0:
        return 1
    else:
        return x * factorial(x - 1)
'''

for s in sp.EncodeAsPieces(unicodedata.normalize('NFKC', text)):
    print(colored(s, random.choice(list(termcolor.COLORS.keys()))), end='')

▁
[構文]
インデントが意味を持つ「オフサイドルール」が特徴的である。

以下に、階乗▁(関数名:▁factorial)を題材にC言語と比較した例を示す。

Pythonのコード:

def▁factorial(x):
▁▁▁▁if▁x▁==▁0:
▁▁▁▁▁▁▁▁return▁1
▁▁▁▁else:
▁▁▁▁▁▁▁▁return▁x▁*▁factorial(x▁-▁1)
